# The Agent Object

The package holds properties of different hazardous chemicals.
The agent object provides usefull functions that use these properties for risk assessments.
Any new risk assessment should first make a new instance of the object with the properties of the requested agent.

## Adding A New Agent

First, we will demonstrate how to add a new agent. This can be done using riskDatalayer.
We have to define the agent name and its description.
An example of an addition of a fictive agent is shown below.
The agent is added by default to a project called "AgentsCollection", but other names may be given.
The agent may also be added to the public database.

In [1]:
from hera import riskassessment as risk
datalayer = risk.riskDatalayer()
from unum.units import *
Agent = "ExampleAgent"
description = {
    "effectParameters" : {
        "tenbergeCoefficient" : 1.5
    },
    "effects" : {
        "RegularPopulation":{
            "type": "Lognormal10",
            "calculator":{
                "TenBerge" : {breathingRate=10*L/min}
            },
            "parameters":{
                "type": "Lognormal10DoseResponse",
                "levels":["Light","Severe"],
                "parameters":{
                "Light": {
                    "TL_50" : 10,
                    "sigma": 0.5
                },
                    "Severe": {
                    "TL_50" : 1,
                    "sigma": 0.5
                }
            }
            }
        },
        "PAC1min":{
            "type":"Threshold",
            "calculator":{
                "MaxConcentration" : {"sampling":"1min"}
            },
            "parameters":{
                "type":"Threshold",
                "levels":["1","2"],
                "parameters":{
                    "1":{"threshold":"1*mg/m**3"},
                    "2":{"threshold":"2*mg/m**3"},
                }
            }
        }
    }
}
#datalayer.addAgent(Agent=Agent,description=description)
#datalayer.addAgent(Agent=Agent,description=description,projectName="otheName") adding the agent to another projec
#datalayer.addAgent(Agent=Agent,description=description, public=True) adding the agent to the public database.

As demonstrated above, each agent has several kinds of effect types, which define different kinds of injuries.
For example, the fictive agent above may use Ten-Berge model in order to assess light and severe injuries, or, on the other hand,
calculate the areas in which the concentration during a 1 minute sampling surpass the defined PAC levels.
When using the agent, one needs to decide which kind of injury type should be used.

As of today, three types of effects are available: Lognormal10, Exponential and Threshold.
Lognormal10 requries values for the parameters "TL_50" and "sigma" for each injury level.
Exponential requries value for the parameter "k" for each injury level.
Threshold requiers value for the parameter "threshold" for each injury level.

Lognormal and exponential distributions of injury levels may use one of two calculators: "Haber" and "TenBerge".
The threshold should use "MaxConcentration". A sampling time must be given to this calculator, as presented above.

## Calculating the toxic load

Some agents use the Ten-Berge model for toxic load. For these agents, the effective toxic load in each point relative to the source may be calculated,
using the agent's appropiate Ten-Berge coefficient. The calculation is done using a concentration field. For example, the calculation for the fictive agent is:

In [2]:
from hera import LSM
sims = LSM.getSimulations(projectName='example')
chosen_sim = sims[0]
concentration = chosen_sim.getConcentration()
Agent = risk.Agent("ExampleAgent")
ToxicLoad = Agent.RegularPopulation.calculateRaw(concentration,"C")

IndexError: list index out of range

## Calculating the effect

The main use of the agent object is to calculate the dimensions of the affected areas for the different injury levels of the different injury types.
This is done simply using the "calculate" function, that should receive a concentration field and the name of the appropiate field in the concentration xarray.

In [3]:
riskAreas = Agent.RegularPopulation.calculate(concentration, "C")

AttributeError: 'str' object has no attribute 'RegularPopulation'

If one desires to calculate the areas for a specific timestep of the concentration xarray, it may be done this way:

In [5]:
import pandas
time = "2018-01-15 10:00:00"
desiredTime = pandas.to_datetime(time)
riskAreas = Agent.RegularPopulation.calculate(concentration, "C", sel={"datetime":desiredTime})
# Or, for the last time step, for example:
riskAreas = Agent.RegularPopulation.calculate(concentration, "C", isel={"datetime":-1})

AttributeError: 'str' object has no attribute 'RegularPopulation'

## Population 

The risk areas calculated in the previous section may be projected on GIS data, such as the population data which exists in hera-data.

In [1]:
from hera import GIS
population = GIS.population(projectName="Examples")
populationAreas = population.projectPolygonOnPopulation(Geometry="TelAviv")
some_angle = 0 # An angle between 0-360
x_coordinate = 178000
y_coordinate = 664000
injuryareas = riskAreas.project(populationAreas, loc=[x_coordinate,y_coordinate],mathematical_angle=some_angle)

KeyboardInterrupt: 